In [198]:
import psycopg2
import pandas as pd
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
import unicodedata
import string
import numpy as np
import spacy    
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import warnings
warnings.filterwarnings("ignore")

In [199]:
def retrieve_reviews(product):
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews where product_name = '{}'".format(product))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df

def retrieve_all_reviews():
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews")
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df



In [153]:
#df = retrieve_reviews('LG Electronics 65UJ6300 65-Inch 4K Ultra HD Smart LED TV (2017 Model)')
df = retrieve_all_reviews()

In [154]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9,10
0,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,1,5.0 out of 5 stars,"September 22, 2017",Prime Customer,"TV Great, TruMotion Bad!",Now that I found the TruMotion setting so that...,Verified Purchase Helpful No...
1,2,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,2,5.0 out of 5 stars,"December 30, 2017",GMC,Hint: Turn OFF Smart Picture Mode,Hint: Turn off Smart Picture Mode and then tur...,Verified Purchase Helpful No...


In [155]:
columns = ['id','p_name','url','p_no','r_no','r_stars','r_date','r_name','r_title','r_text','r_comments']

In [156]:
df.columns = columns

In [157]:
df.head(2)

,id,p_name,url,p_no,r_no,r_stars,r_date,r_name,r_title,r_text,r_comments
0,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,1,5.0 out of 5 stars,"September 22, 2017",Prime Customer,"TV Great, TruMotion Bad!",Now that I found the TruMotion setting so that...,Verified Purchase Helpful No...
1,2,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,2,5.0 out of 5 stars,"December 30, 2017",GMC,Hint: Turn OFF Smart Picture Mode,Hint: Turn off Smart Picture Mode and then tur...,Verified Purchase Helpful No...


In [158]:
df.groupby(['r_stars']).count()

,id,p_name,url,p_no,r_no,r_date,r_name,r_title,r_text,r_comments
r_stars,,,,,,,,,,
1.0 out of 5 stars,650,650,650,650,650,650,650,650,650,650
2.0 out of 5 stars,229,229,229,229,229,229,229,229,229,229
3.0 out of 5 stars,275,275,275,275,275,275,275,275,275,275
4.0 out of 5 stars,591,591,591,591,591,591,591,591,591,591
5.0 out of 5 stars,2867,2867,2867,2867,2867,2867,2867,2867,2867,2867


In [161]:
df_new = df.copy()
#df_new = df_new[df_new['r_stars'].apply(lambda x: x in ['5.0 out of 5 stars'])]
df_new['rating'] = df_new['r_stars'].apply(lambda x: 1 if x == '5.0 out of 5 stars' else 0)
df_new = df_new[['rating','r_name','r_text']]
df_new.head(3)

,rating,r_name,r_text
0,1,Prime Customer,Now that I found the TruMotion setting so that...
1,1,GMC,Hint: Turn off Smart Picture Mode and then tur...
2,1,AncientGhost,Needed an upgrade for my 7yr old HDTV and deci...


In [162]:
nlp = spacy.load("en")
document = df_new.iloc[2]['r_text']
parsed_document = nlp(document)

In [163]:
print (parsed_document)

Needed an upgrade for my 7yr old HDTV and decided to get the 43UJ6300. Going from my old tv to this one is like night and day. I have an Xbox One S and after I turned on the HDR in the settings Mass Effect Andromeda looks AMAZING!! Ive watched some 4K videos on youtube and it feels like you are there in the video. I highly recommend this tv for anyone looking to upgrade to 4K. I do have one regret, I wish Id purchased the 49 inch version but its cool. If youre looking for an entry level 4KTV at a great price I would give this tv a chance. I think you will be very pleased. I know I am!!


In [164]:
for num, sentence in enumerate(parsed_document.sents):
    print (num, sentence)

0 Needed an upgrade for my 7yr old HDTV and decided to get the 43UJ6300.
1 Going from my old tv to this one is like night and day.
2 I have an Xbox One S and after I turned on the HDR in the settings Mass Effect Andromeda looks AMAZING!!
3 Ive watched some 4K videos on youtube and it feels like you are there in the video.
4 I highly recommend this tv for anyone looking to upgrade to 4K.
5 I do have one regret, I wish Id purchased the 49 inch version but its cool.
6 If youre looking for an entry level 4KTV at a great price I would give this tv a chance.
7 I think you will be very pleased.
8 I know I am!!


In [165]:
for num, entity in enumerate(parsed_document.ents):
    print (entity, entity.label_)
token_text = [token.orth_ for token in parsed_document]
token_pos = [token.pos_ for token in parsed_document]

7yr ORDINAL
Xbox One S ORG
HDR ORG
youtube PRODUCT
49 inch QUANTITY
4KTV CARDINAL


In [166]:
token_lemma = [token.lemma_ for token in parsed_document]
token_shape = [token.shape_ for token in parsed_document]
token_prob = [token.prob for token in parsed_document]

In [167]:
df_spicy = pd.DataFrame({'token_text':token_text, 'token_lemma':token_lemma, 'token_shape':token_shape,'token_prob':token_prob})

In [168]:
stop_words = set(stopwords.words('english'))

def punct_space(token):
    return token.is_punct or token.is_space

def is_stop_word(token):
    return token in stop_words

def lemmatized_sentence_corpus (documents):
    for doc in documents:
        parsed_doc = nlp(doc)
        for sent in parsed_doc.sents:
            yield (' '.join([token.lemma_ for token in sent if not punct_space(token)]))


In [169]:
bigram_model = Phrases([doc.split(" ") for doc in lemmatized_sentence_corpus(df_new['r_text'].values)], min_count=1)
bigram_sentences = []
for unigram_sentence in lemmatized_sentence_corpus(df_new['r_text'].values):
    bigram_sentences.append(' '.join(bigram_model[unigram_sentence.split(" ")]))
trigram_model = Phrases([doc.split(" ") for doc in bigram_sentences], min_count=1)
type(trigram_model.vocab)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


collections.defaultdict

In [170]:
' '.join(bigram_model['not','with'])

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


'not with'

In [169]:
# from gensim.models import Phrases
# documents = ["the mayor of york mayor was there", "machine learning can be useful sometimes","new york mayor was present"]

# sentence_stream = [doc.split(" ") for doc in documents]
# print (sentence_stream)
# bigram = Phrases(sentence_stream, min_count=1, threshold=1)
# sent = [u'the', u'of', u'new', u'york',  u'mayor', u'was', u'there']
# print(bigram[sent])

In [171]:
bigram_sentences[0]

trigram_sentences = []
for bigram_sentence in bigram_sentences:
    trigram_sentences.append(' '.join(trigram_model[bigram_sentence.split(" ")]))

trigram_sentences[6]

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


'hint turn_off smart picture_mode and then turn the energy_saver off. with smart picture_mode on and no access to energy_saver setting the picture be very dull and drab'

In [219]:
# trigram_reviews = LineSentence(trigram_sentences)
# trigram_dictionary = Dictionary([doc.split(" ") for doc in trigram_sentences])
# trigram_dictionary.filter_extremes(no_below=5, no_above=0.4)
# def trigram_bow_generator(trigram_sentences):
#     trigram_bow = []
#     for review in trigram_sentences:
#         print (review)
#         trigram_bow.append(trigram_dictionary.doc2bow(review))

In [26]:
#trigram_bow = trigram_bow_generator([doc.split(" ") for doc in trigram_sentences])

In [172]:
corpus = []
terms_doc = []
count = 0
for doc in df_new['r_text'].values:
    parsed_doc = nlp(doc)
    bigram_doc = ' '.join(bigram_model[(token.lemma_ for token in parsed_doc if not punct_space(token))])
    trigram_doc = ' '.join(trigram_model[(token for token in bigram_doc.split(" ") if not is_stop_word(token))])    
    corpus.append (trigram_doc)
    terms_doc.append(trigram_model[(token for token in bigram_doc.split(" ") if not is_stop_word(token))])

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [173]:
data = corpus

In [200]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(corpus)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
 
# Let's see how the first document in the corpus looks like in different topic spaces
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])

(4612, 10)
(4612, 10)
(4612, 10)
[0.0026317  0.00263181 0.00263175 0.00263158 0.00263183 0.89025453
 0.00263158 0.00263158 0.00263191 0.08869172]
[0.02655959 0.09844239 0.         0.         0.         0.
 0.         0.16333788 0.25418854 0.        ]
[ 2.58052253  0.94103156 -1.66686151 -1.01942557  0.57124486 -0.48279677
  0.57483933  1.13645792  1.17151518 -1.32079813]


In [176]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)
 
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('color', 657.782973077081), ('picture', 478.5436741954339), ('hdr', 452.4794167057157), ('look', 421.3616555857783), ('qled', 335.76595594116117), ('samsung', 317.01051575044085), ('best', 271.2841461707914), ('quality', 252.3587064920658), ('bright', 243.7694023776348), ('game', 238.44303808365595)]
Topic 1:
[('use', 1071.4886694294516), ('like', 731.3295892699615), ('cable', 700.942306071483), ('connect', 681.6150364659843), ('picture', 634.1152862489782), ('box', 609.6435297763688), ('smart', 490.9798141405895), ('set', 477.05543104605835), ('wall', 449.6094049122344), ('need', 447.28521067459053)]
Topic 2:
[('great', 1513.2991214238468), ('picture', 1357.0071421145117), ('love', 678.9477448684993), ('sound', 631.7040552862427), ('good', 579.277955342291), ('quality', 523.6628043068524), ('easy', 443.91892624554816), ('price', 353.70302139290243), ('set', 297.33492690824863), ('excellent', 268.09938090032335)]
Topic 3:
[('work', 786.0735718377712), ('problem', 

In [177]:
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [201]:
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(corpus))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [202]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(corpus)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "The economy is working better than ever"
x = lda_model.transform(vectorizer.transform([text]))[0]
print(x, x.sum())

[0.36666676 0.03333967 0.03333761 0.03333407 0.03333333 0.03333334
 0.03333336 0.03333333 0.36665519 0.03333333] 1.0000000000000002


In [203]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
1      37.084017        1       1  133.464920  215.676743
8      13.752328        1       2 -237.102921   21.516123
3      12.895038        1       3   17.676514 -275.262329
6      12.213358        1       4  -76.967262  -60.473282
2       8.479292        1       5   91.164597 -105.530571
4       6.240617        1       6  241.399734 -198.351883
5       5.494645        1       7   45.965969   62.352093
0       2.088005        1       8 -179.034973 -208.276001
7       1.125177        1       9  244.579208   24.191263
9       0.627523        1      10  -99.309746  176.149414, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
900   Default  2024.000000        great  2024.000000  30.0000  30.0000
1471  Default  2530.000000      picture  2530.000000  29.0000  29.0000
2192  Default  1554.000000          use  1554.000000  28.0000  28.0000
1197  Default   814.000000         love   814.000000  27.0000  27.0000
1809  Default  1128.000000          set  1128.000000  26.0000  26.0000
1752  Default  2241.000000      samsung  2241.000000  25.0000  25.0000
642   Default   717.000000         easy   717.000000  24.0000  24.0000
1907  Default  1044.000000        sound  1044.000000  23.0000  23.0000
1591  Default  1090.000000      quality  1090.000000  22.0000  22.0000
2043  Default   532.000000   television   532.000000  21.0000  21.0000
1679  Default  1600.000000       remote  1600.000000  20.0000  20.0000
888   Default   977.000000         good   977.000000  19.0000  19.0000
550   Default   302.000000       design   302.000000  18.0000  18.0000
2296  Default  1227.000000         work  1227.000000  17.0000  17.0000
250   Default   821.000000          box   821.000000  16.0000  16.0000
1538  Default   669.000000        price   669.000000  15.0000  15.0000
1189  Default  1197.000000         look  1197.000000  14.0000  14.0000
104   Default   344.000000          app   344.000000  13.0000  13.0000
367   Default   741.000000        color   741.000000  12.0000  12.0000
940   Default   438.000000          hdr   438.000000  11.0000  11.0000
386   Default   180.000000      company   180.000000  10.0000  10.0000
1776  Default   998.000000       screen   998.000000   9.0000   9.0000
2005  Default   323.000000      support   323.000000   8.0000   8.0000
508   Default   501.000000          day   501.000000   7.0000   7.0000
2134  Default   249.000000        tuner   249.000000   6.0000   6.0000
1090  Default   568.000000        issue   568.000000   5.0000   5.0000
223   Default   477.000000        black   477.000000   4.0000   4.0000
2243  Default   243.000000     warranty   243.000000   3.0000   3.0000
1004  Default   233.000000        image   233.000000   2.0000   2.0000
1556  Default   417.000000      product   417.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
1702  Topic10    22.191874  requirement    23.018890   5.0346  -3.6328
788   Topic10    20.898798    fireplace    21.730728   5.0321  -3.6928
1580  Topic10    19.596495        prove    20.423489   5.0298  -3.7572
2265  Topic10    18.444641   whatsoever    19.272056   5.0273  -3.8177
2021  Topic10    17.286214        sweet    18.113667   5.0244  -3.8826
1240  Topic10    17.017440          max    17.844493   5.0237  -3.8983
2197  Topic10    16.878786        usual    17.713814   5.0229  -3.9065
1963  Topic10    16.260883       stress    17.092208   5.0213  -3.9438
1314  Topic10    16.139850   navigation    16.966848   5.0212  -3.9512
405   Topic10    15.881908   compliment    16.708924   5.0204  -3.9673
669   Topic10    15.612635  enhancement    16.439961   5.0195  -3.9844
1445  Topic10    13.022368       pencil    13.849551   5.0096  -4.1659
787   Topic10    17.894342         fios    19.034575   5.0094  -3.

In [181]:
from gensim.models import Word2Vec

In [182]:
model = Word2Vec(terms_doc, size=100, window=5, min_count=5, workers=4)

In [197]:
model.wv.most_similar(positive=['very_frustrating','lg'])
#model.wv.index2word

[('since', 0.997528076171875),
 ('help', 0.997459888458252),
 ('day', 0.9974228739738464),
 ('call_samsung', 0.9974119663238525),
 ('may', 0.9973971247673035),
 ('refund', 0.9973798394203186),
 ('another', 0.9973711371421814),
 ('come', 0.9973693490028381),
 ('point', 0.9973652958869934),
 ('submit', 0.9973627328872681)]

In [147]:
print ('' in stop_words)

False
